In [ ]:
import os
import random
import pandas
import importlib  # used to reload modules if necessary
path_to_frontend = '..'
os.chdir(path_to_frontend)
print('current working directory:', os.getcwd())
import q100viz.session as session
import q100viz.devtools as devtools
import q100viz.graphics.graphs as graphs

In [ ]:
session.buildings.df['selected'] = False
session.buildings.df['group'] = -1

In [ ]:
# prepare simulation:

devtools.mark_random_buildings_for_simulation(session.buildings.df, 4, connection_to_heat_grid=random.randint(2020,2040), refurbished=True)


In [ ]:
# start simulation:
session.simulation.setup(input_max_year=2045)
session.simulation.run()

In [ ]:
# test batch-simulation:
session.VERBOSE_MODE = True
importlib.reload(graphs)
# 1. 2 specific + 2 random buildings, no decisions
devtools.select_buildings_for_simulation(session.buildings.df, ['5.11', '6.06'])
devtools.select_random_buildings_for_simulation(
            session.buildings.df, 2)
session.simulation.setup(input_max_year=2022)
session.simulation.run()

# 2. same buildings, connection=2022
devtools.select_random_buildings_for_simulation(
            session.buildings.df[session.buildings.df['selected'] == True], 4, connection_to_heat_grid=2021)
session.simulation.setup(input_max_year=2022)
session.simulation.run()

# 2. same buildings, connection=2022 + refurbished
devtools.select_random_buildings_for_simulation(
            session.buildings.df[session.buildings.df['selected'] == True], 4, connection_to_heat_grid=2021, refurbished=True)
session.simulation.setup(input_max_year=2022)
session.simulation.run()

# 2. same buildings, connection=2022 + save_energy
session.buildings.df['refurbished'] = False  # reset refurbishment

devtools.select_random_buildings_for_simulation(
            session.buildings.df[session.buildings.df['selected'] == True], 4, connection_to_heat_grid=2021, save_energy=True)
session.simulation.setup(input_max_year=2022)
session.simulation.run()

## single graph design

In [ ]:
importlib.reload(graphs)
# for group_df in session.buildings.list_from_groups():
#     if group_df is not None:
#         for idx in group_df.index:
idx = "1.02"

output_folders = ["../data/outputs/output_test"]
current_output_folder = output_folders[0]

# export emissions graph:
graphs.export_individual_emissions(
    csv_name="/emissions/CO2_emissions_{0}.csv".format(
        idx),
    data_folders=output_folders,
    columns=['building_household_emissions'],
    title_="Emissionen",
    # outfile=current_output_folder +
    # "/emissions/CO2_emissions_{0}.png".format(idx),
    xlabel_="Jahr",
    ylabel_="$CO_{2}$-Äquivalente (kg/Monat)",  # TODO: t/Jahr
    x_='current_date',
    convert_grams_to_kg=True,
    compare_data_folder=current_output_folder + "/../../precomputed/simulation_defaults",
    figtext=
        str(idx) + " "
        + str(group_df.loc[idx, 'address']) + " "
        + str(group_df.loc[idx, 'type'])
        + "\nø-spez. Wärmeverbrauch: "
        + str(group_df.loc[idx, 'avg_spec_heat_consumption'])
        + ", ø-spez. Stromverbrauch: "
        + str(group_df.loc[idx, 'avg_spec_heat_consumption'])
        if session.VERBOSE_MODE else "",
    figsize=(16,12)  # inches
)

# export energy prices graph:
graphs.export_individual_energy_expenses(
    csv_name="/energy_prices/energy_prices_{0}.csv".format(
        idx),
    data_folders=output_folders,
    columns=['building_household_expenses_heat',
            'building_household_expenses_power'],
    labels_=['Wärmekosten', 'Stromkosten'],
    # outfile=current_output_folder +
    # "/energy_prices/energy_prices_{0}.png".format(idx),
    title_="Energiekosten",
    xlabel_="Jahr",
    ylabel_="€/Monat",
    x_='current_date',
    compare_data_folder=current_output_folder + "/../../precomputed/simulation_defaults",
    figtext=
        str(idx) + " "
        + str(group_df.loc[idx, 'address']) + " "
        + str(group_df.loc[idx, 'type'])
        + "\nø-spez. Wärmeverbrauch: "
        + str(group_df.loc[idx, 'avg_spec_heat_consumption'])
        + ", ø-spez. Stromverbrauch: "
        + str(group_df.loc[idx, 'avg_spec_heat_consumption'])
        if session.VERBOSE_MODE else "",
    figsize=(16,12),  # inches
    prepend_historic_data=True
)

## neighborhood graphs

In [ ]:
# neighborhood total emissions:
importlib.reload(graphs)
graphs.export_individual_graph(
    csv_name="/emissions/CO2_emissions_neighborhood.csv",
    data_folders=output_folders,
    columns=['emissions_neighborhood_accu'],
    title_="jährlich kumulierte Gesamtemissionen des Quartiers",
    # outfile=current_output_folder + "/emissions/CO2_emissions_neighborhood.png",
    xlabel_="Jahr",
    ylabel_="$CO_{2}$-Äquivalente (t)",
    x_='current_date',
    convert_grams_to_tons=True,
    compare_data_folder=current_output_folder + "/../../precomputed/simulation_defaults"
)

# neighborhood total energy prices prognosis:
graphs.export_neighborhood_total_data(
    csv_name="/energy_prices/energy_prices_total.csv",
    data_folders=[current_output_folder],
    columns=['gas_price', 'power_price', 'oil_price'],
    labels_=['Gaspreis', 'Strompreis', 'Ölpreis',],
    title_="generelle Energiepreise nach Energieträger",
    # outfile=current_output_folder + "/energy_prices/energy_prices_total.png",
    xlabel_="Jahr",
    ylabel_="Preis (ct/kWh)",
    x_='current_date',
    label_show_iteration_round=False
    # compare_data_folder=current_output_folder + "/../../precomputed/simulation_defaults"
)

In [ ]:

# export energy prices graph:
# importlib.reload(graphs)
# session.simulation.output_folders = ["/home/dunland/github/qScope/data/outputs/output_20221025_13-56-59", "/home/dunland/github/qScope/data/outputs/output_20221025_14-08-17"] 
for group_df in session.buildings.list_from_groups():
    if group_df is not None:
        for idx in group_df.index:
            graphs.export_individual_graph(
                csv_name="/energy_prices/energy_prices_{0}.csv".format(
                    idx),
                data_folders=["/home/dunland/github/qScope/data/outputs/output_test"],
                columns=['building_household_expenses_heat',
                            'building_household_expenses_power'],
                labels_=['Wärmekosten', 'Stromkosten'],
                outfile=None,
                title_="Energiekosten",
                xlabel_="Jahr",
                ylabel_="€/Monat",
                x_='current_date',
                compare_data_folder=session.simulation.reference_data_folder,
                figtext=
                    str(idx) + " "
                    + str(group_df.loc[idx, 'address']) + " "
                    + str(group_df.loc[idx, 'type'])
                    + "\nø-spez. Wärmeverbrauch: "
                    + str(group_df.loc[idx, 'avg_spec_heat_consumption'])
                    + ", ø-spez. Stromverbrauch: "
                    + str(group_df.loc[idx, 'avg_spec_heat_consumption'])
                    if session.VERBOSE_MODE else "",
                figsize=(16,12)  # inches
            )

## combined graphs

In [ ]:
session.simulation.current_output_folder = "../data/outputs/output_test/"
devtools.select_buildings_for_simulation(session.buildings.df, ['1.02', '5.11', '6.06', '2.07'], connection_to_heat_grid=2027, refurbished=True)

# combined emissions graph for selected buildings:
graphs.export_compared_emissions(
    session.buildings.list_from_groups(),
    session.simulation.current_output_folder
    )

# combined energy prices graph for selected buildings:
graphs.export_compared_energy_costs(
    search_in_folder=session.simulation.current_output_folder)

In [ ]:
importlib.reload(session)
# create graphs from precomputed reference data
for idx in session.buildings.df.index:
    graphs.export_default_graph(
        csv_name="/energy_prices/energy_prices_{0}.csv".format(
            idx),
        data_folders=[session.simulation.reference_data_folder],
        csv_columns=['building_household_expenses_heat',
                    'building_household_expenses_power'],
        labels_=['Wärmekosten', 'Stromkosten'],
        outfile=session.simulation.reference_data_folder + "/energy_prices/",
        title_="Energiekosten",
        xlabel_="Jahr",
        ylabel_="€/Monat",
        x_='current_date',
        # compare_data_folder=session.simulation.current_output_folder + "/../../precomputed/simulation_defaults",
        figsize=(16,12),  # inches
        overwrite_color='lightgray'
    )

In [ ]:
importlib.reload(graphs)

devtools.select_buildings_for_simulation(session.buildings.df, ['1.02'], connection_to_heat_grid=2027, refurbished=True)

session.buildings.df.at['1.02', 'group'] = 2
print(session.buildings.df.loc['1.02', 'group'])

current_output_folder = "/home/dunland/github/qScope/data/outputs/output_test"
graphs.export_compared_emissions(
            session.buildings.list_from_groups(),
            current_output_folder,
            current_output_folder + "/emissions/CO2_emissions_groups.png"
            )

In [ ]:
df_prepend_expenses=pandas.read_csv("../data/data_pre-simulation/energy-expenses_hh_2000-2020.csv")
output_folder = "../data/outputs/output_test/"
csv_name = "/energy_prices/energy_prices_1.02.csv"

df_prepend_expenses = df_prepend_expenses.rename(columns={
    'year' : 'current_date',
    'hh_heat_expenses_2000_2020' : 'building_household_expenses_heat',
    'hh_power_expenses_2000_2020' : 'building_household_expenses_power'
})

df_prepend_expenses = df_prepend_expenses[df_prepend_expenses['current_date'] < 2020]

csv_data = pandas.read_csv(output_folder + csv_name)
csv_data['current_date'] = csv_data['current_date'].apply(graphs.GAMA_time_to_datetime)

csv_data = pandas.concat([df_prepend_expenses, csv_data])
print(csv_data)